In [282]:
# Using PyTorch for NN
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
import inputparser


In [283]:
input_vector, y_test = inputparser.make_input()

In [284]:
# x-test, y-test are 225 data points, remove first element for verification
x_verification = []
x_verification.append(input_vector[0])
y_verification = y_test[0]

x_input = input_vector[1:]
y_input = []
for i in y_test[1:]:
    y_input.append(i)

In [285]:
# Random seeding
torch.manual_seed(44)
np.random.seed(44)

# Input data
X = torch.tensor(x_input.tolist())
Y = torch.tensor(y_input)

# Split data
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# NN
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize
input_size = len(x_input[0])
# Optimal hidden layer size
hidden_size = 290  
model = NeuralNet(input_size, hidden_size)

# loss
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(141):
    # forward prop
    outputs = model(torch.tensor(X_train, dtype=torch.float32))
    loss = criterion(outputs.squeeze(), Y_train)
    # back prop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/141], Loss: {loss.item():.4f}')
print("Final Epoch Count:", epoch + 1)

with torch.no_grad():
    train_outputs = model(torch.tensor(X_train, dtype=torch.float32))
    train_mse = mean_squared_error(train_outputs.squeeze().detach().numpy(), Y_train)
print("Training MSE:", train_mse)

with torch.no_grad():
    val_outputs = model(torch.tensor(X_val, dtype=torch.float32))
    val_mse = mean_squared_error(val_outputs.squeeze().detach().numpy(), Y_val)
print("Validation MSE:", val_mse)


Epoch [100/141], Loss: 0.0056
Final Epoch Count: 141
Training MSE: 0.003620665
Validation MSE: 0.017644884
